# 微分可能LUTモデルによるCNNでのCIFAR-10学習

Differentiable LUTモデルで畳み込み層を形成して、一般的なデータに対してCNNによる回路学習を行います。 

In [1]:
import os
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

### データセット

データセットの準備には torchvision を使います

In [2]:
# configuration
net_name              = 'Cifar10DifferentiableLutCnn'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/cifar10'
rtl_module_name       = 'Cifar10LutCnn'
output_velilog_file   = os.path.join(data_path, rtl_module_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

bin_mode              = True
frame_modulation_size = 7
epochs                = 8
mini_batch_size       = 32

# dataset
dataset_train = torchvision.datasets.CIFAR10(root=data_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.CIFAR10(root=data_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

Extracting ./data/Cifar10DifferentiableLutCnn\cifar-10-python.tar.gz to ./data/Cifar10DifferentiableLutCnn
Files already downloaded and verified


## ネットワーク構築

Convolution2d を使って畳み込み層を作ります。<br>
Convolution2d は指定した層を im2col と col2im で挟み込んで Lowering による畳み込みをサポートします。<br>
DenseAffine を Lowering すると一般にCNNで知られる畳み込み層になりますが、LUT-Network では
ここに DifferentiableLut を組み合わせて作った層を設定することでDenseAffineとは異なる効率の良い畳み込み層を実現します。

In [3]:
# バイナリ時は BIT型を使えばメモリ削減可能
bin_dtype = bb.DType.BIT if bin_mode else bb.DType.FP32

# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size, depth_modulation_size=15, bin_dtype=bin_dtype),
            bb.Sequential([
                bb.Convolution2d(  # pointwise
                    bb.Sequential([
                        bb.DifferentiableLut([512], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([32*6], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([32], bin_dtype=bin_dtype),
                    ]),
                    filter_size=(1, 1),
                    fw_dtype=bin_dtype),

                bb.Convolution2d(  # 32x32 -> 30x30
                    bb.Sequential([
                        bb.DifferentiableLut([512], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([32*6], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([32], bin_dtype=bin_dtype),
                    ]),
                    filter_size=(3, 3),
                    fw_dtype=bin_dtype),
                bb.Convolution2d(  # 30x30 -> 28x28
                    bb.Sequential([
                        bb.DifferentiableLut([1024], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([64*6], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([64], bin_dtype=bin_dtype),
                    ]),
                    filter_size=(3, 3),
                    fw_dtype=bin_dtype),
                bb.MaxPooling(filter_size=(2, 2), fw_dtype=bin_dtype),  # 28x28-> 14x14
            ]),
            bb.Sequential([
                bb.Convolution2d(  # 14x14-> 12x12
                    bb.Sequential([
                        bb.DifferentiableLut([1024], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([64*6], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([64], bin_dtype=bin_dtype),
                    ]),
                    filter_size=(3, 3),
                    fw_dtype=bin_dtype),
                bb.Convolution2d(  # 12x12-> 10x10
                    bb.Sequential([
                        bb.DifferentiableLut([2048], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([128*6], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([128], bin_dtype=bin_dtype),
                    ]),
                    filter_size=(3, 3),
                    fw_dtype=bin_dtype),
                bb.MaxPooling(filter_size=(2, 2), fw_dtype=bin_dtype),  # 10x10-> 5x5
            ]),
            bb.Sequential([
                bb.Convolution2d(  # 5x5-> 1x1
                    bb.Sequential([
                        bb.DifferentiableLut([1024*6],   bin_dtype=bin_dtype),
                        bb.DifferentiableLut([1024],     bin_dtype=bin_dtype),
                        bb.DifferentiableLut([10*6*6*6], bin_dtype=bin_dtype),
                        bb.DifferentiableLut([10*6*6],   bin_dtype=bin_dtype),
                        bb.DifferentiableLut([10*6],     bin_dtype=bin_dtype),
                        bb.DifferentiableLut([10],       bin_dtype=bin_dtype),
                    ]),
                    filter_size=(5, 5),
                    fw_dtype=bin_dtype),
            ]),
            bb.BinaryToReal(frame_modulation_size=frame_modulation_size, bin_dtype=bin_dtype)
        ])

net.set_input_shape([3, 32, 32])

if bin_mode:
    net.send_command("binary true")

# print(net.get_info()) # ネットを表示

## 学習実施

学習を行います

In [ ]:
#bb.load_networks(data_path, net)

# learning
loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

for epoch in range(epochs):
    loss.clear()
    metrics.clear()

    # learning
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)

            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            net.backward(dy_buf)

            optimizer.update()

            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    bb.save_networks(data_path, net)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

## RTL(Verilog)変換

FPGA化するために Verilog に変換します。インターフェースはXilinx社のAXI4 Stream Video 仕様(フレームスタートでtuserが立つ)となります。
MaxPooling の単位で画像サイズが縮小されてしまうので、現状、この単位でしか変換できないため3つに分けて出力しています。

In [ ]:
# export verilog
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_cnv_layers(f, rtl_module_name + 'Cnv0', net[1])
    bb.dump_verilog_lut_cnv_layers(f, rtl_module_name + 'Cnv1', net[2])
    bb.dump_verilog_lut_cnv_layers(f, rtl_module_name + 'Cnv2', net[3])